# Electric Vehicle Charge Scheduling MDP

### Dependencies

You will need to install POMDPs and POMDPToolbox locally before being able to run this notebook. This can be done by running the following in your local Julia 1.0 REPL: 
    - Pkg.add("POMDPs")
    - Pkg.add("POMDPModelTools")
    - Pkg.add("POMDPSimulators")

In [1]:
using POMDPs, POMDPModelTools, POMDPSimulators, Random

### State Structure
Define state structure / make initial constructor

In [2]:
mutable struct evState
    p::Vector{Bool} # array of whether cars are present
    c::Vector{Int64} # array of charge in each car
    renew::Int64 # renewable energy level
    t::Int64 # time
    done::Bool # are we in a terminal state
end

# initial state constructor
evState(p,c,renew::Int64,t::Int64) = evState(p,c,renew,t,false)

evState

### MDP Structure
Define MDP structure with everything you would need / make initial constructor

In [3]:
struct evMDP <: MDP{evState,Vector{Bool}} 
    n::Int64 # number of cars
    T::Int64 # number of timesteps
    renew_levels::Int64 # number of renewable mixture levels, 0:renew_levels
    charge_levels::Int64 # number of charge levels, 0:charge_levels
    λ::Float64 # terminal reward weighting
end

# we use key worded arguments so we can change any of the values we pass in 
function evMDP(;n::Int64 = 3, # number of cars
                T::Int64 = 6, # number of timesteps
                renew_levels::Int64 = 4, # number of renewable mixture levels, 0:renew_levels
                charge_levels::Int64 = 4, # number of charge levels, 0:charge_levels
                λ::Float64 = 100) #terminal reward weighting
    return evMDP(n, T, renew_levels, charge_levels, λ)
end


evMDP

### States
Define all possible states

In [4]:
# convert number to an array of numbers using requested base system, array length
function num2array(number,base,array_length)
    base==2 ? finalarray = zeros(Bool, array_length) : finalarray = zeros(Int64, array_length)
    idx=1
    while number > 0
        finalarray[idx] = rem(number,base)
        number = div(number,base)
        idx+=1
    end
    return finalarray
end

function POMDPs.states(mdp::evMDP)
    s = GridWorldState[] # initialize an array of GridWorldStates
    
    # add every possible state. This includes every possible combination of present/charge array
    
    for iP = 0:(2^mdp.n-1)
        present = num2array(iP,2,mdp.n)
        
        for iC = 0:((mdp.charge_levels+1)^mdp.n-1)
            charge = num2array(iC,(mdp.charge_levels+1),mdp.n)
            
            for rl=0:mdp.renew_levels, t=1:mdp.T
            
                # if in final time, make sure the done flag is set on
                t==mdp.T ? push!(s,evState(present, charge, rl, t, true)) : push!(s,evState(present, charge, rl, t)) 
            end
        end
    end
    return s
end


In [5]:
POMDPs.initialstate(mdp::evMDP, rng::AbstractRNG) = evState(zeros(Bool,mdp.n),zeros(Int64,mdp.n), mdp.renew_levels, 1)

### Actions
Define all possible action vectors

In [6]:
function POMDPs.actions(mdp::evMDP)
    # initialize empty action space a
    a = []    
    # populate with all combinations of actions, ex [true, false, true, true]
    for iA=0:(2^mdp.n-1)
        push!(a,num2array(iA,2,mdp.n))
    end
    return a
end
    

### Reward Function
Define the reward function

In [7]:
function POMDPs.reward(mdp::evMDP, state::evState, action::Vector{Bool}, statep::evState)
    r = mdp.lambda*mdp.charge_levels 
    if state.done
        max_c = mdp.charge_levels
        for i in 1:length(state.p)
            if state.p[i] == true
                r -= exp((max_c-state.c[i])/max_c)
            end
        end
    end
    return r
end

### Transition Function
Define the next-state transition probabilities (this is the hard one)

In [8]:

# chance of a car appearing in any slot at time t given total timescale T
carAppearProb(t,T) = 1/(1+exp(-20(t-2)/T))

# function takes a state to generate different charge probabilities for, the previous presence array, the mdp, 
# and the probability of transitioning to 
function carAppearStates(baseState,prevP,mdp,probability)
    
    newCarIdxs = findall(baseState.p==prevP)

    probs = zeros(Int64,mdp.charge_levels^length(newCarIdxs)) .+ probability/(mdp.charge_levels^length(newCarIdxs))
    
    chargeCombs = [baseState.c]
    for ind in newCarIdxs
        chargeCombsPrev = chargeCombs
        
        chargeCombs = []
        for charge in chargeCombsPrev, level in 0:(mdp.charge_levels-1)
            addition = charge[:]
            addition[ind] = level
            push!(chargeCombs,addition)
        end
    end
    
    state_vec = evState[]
    for charge in chargeCombs
        push!(state_vec,evState(baseState.p, charge, baseState.renew, baseState.t, baseState.done))
    end
    return SparseCat(state_vec, probs)
end
    
    

carAppearStates (generic function with 1 method)

In [9]:
prevP = [true, false, true, false]

newP = [true, true, true, true]
newC = [3, 4, 4, 4]
baseState = evState(newP,newC,0,1,false)
test_mdp = evMDP(4,4,4,4,0.1)

carAppearStates(baseState,prevP,test_mdp,1.0)


SparseCat{Array{evState,1},Array{Float64,1}}(evState[evState(Bool[true, true, true, true], [3, 4, 4, 4], 0, 1, false)], [1.0])

In [49]:
calc_renew_delt(t) = 0

function POMDPs.transition(mdp::evMDP, state::evState, action::Vector{Bool})
    t_next = state.t + 1
    if t_next == mdp.T
        done_bool = true
    end
    delta_charge = 0.25*4/mdp.renew_levels# Amount of energy lost per charge per car as a fraction of total charge level
    renew_next = round.(Int8,state.renew + (calc_renew_delt(t) - length(findall(action))*delta_charge)*mdp.renew_levels)
    c_next = round.(Int8, state.c + action*delta_charge*mdp.renew_levels)
    # print(state.p)
    wherenocars = findall(iszero,state.p)
    p_next = [state.p]
    num_new_cars = [0]
    num_initial_spaces = length(wherenocars)
    for i in 1:num_initial_spaces
        for p in p_next
            println(p)
            state_new = p[:]
            println(wherenocars[i])
            println(state_new)
            state_new[wherenocars[i]] = true
            push!(p_next,state_new)
        end
    end
    for p in p_next
        n_new_cars = length(find(p & !state.p))
        push!(num_new_cars,n_new_cars)
    end
    car_prob = carAppearProb(t_next,mdp.T) 
    probs = [(car_prob^n)*(1-car_prob)^(num_initial_spaces-n) for n in num_new_cars]
    next_states = [evState(p_new,c_next,t_next,renew_next,done_bool) for p_new in p_next]
    return SparseCat(next_states,probs)
end

In [40]:
j = [true, true, false, false]
print(findall(j))

[3, 4]

In [ ]:
test_mdp = evMDP(4,4,4,4,0.1)


P = [true, false, true, false]
action = [true, true, true, true]
C = [3, 4, 4, 4]
baseState = evState(P,C,0,1,false)

POMDPs.transition(test_mdp,baseState,action)


Bool[true, false, true, false]
2
Bool[true, false, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[t

Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[tru

2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[t

Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[tru

Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[tru

...

Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[tru

Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[tru

Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[true, true, true, false]
2
Bool[true, true, true, false]
Bool[tru

Excessive output truncated after 524289 bytes.

 true, false]
Bool[true,

### Miscellaneous Functions
Define other functions that POMDPs.jl needs

In [11]:
POMDPs.n_states(mdp::evMDP) = 2^mdp.n*(mdp.charge_levels+1)^mdp.n*(mdp.renew_levels+1)*mdp.T
POMDPs.n_actions(mdp::evMDP) = 2^mdp.n
POMDPs.discount(mdp::evMDP) = 1
POMDPs.isterminal(mdp::evMDP, s::evState) = s.done

In [12]:
# define state and action indexing
function indVal(base, a)
    ind = 1
    for i in 1:length(a)
        ind += a[i]*base^(i-1)
    end
    return ind
end    

function POMDPs.stateindex(mdp::evMDP, state::evState)
    indP = indVal(2,state.p)
    indC = indVal(mdp.charge_levels,state.c)
    indR = state.renew + 1
    indT = state.t
    maxP = 2^(mdp.n)
    maxC = (mdp.charge_levels + 1)^(mdp.n)
    maxR = mdp.renew_levels + 1
    maxT = mdp.T
    sInd = indP + (maxP*(indC-1)) + (maxC*maxP*(indR-1)) + (maxR*maxC*maxP*(indT-1))
    return sInd
end

function POMDPs.actionindex(mdp::evMDP, act::Vector{Bool})
    return indVal(2,act)
end

In [13]:
mdp1 = evMDP(3,2,5,5,0.1)
state1 = evState([1,0,0],[1,0,0],0,1,false)

println(POMDPs.stateindex(mdp1,state1))


10


### Implement Solvers / Simulators

In [14]:
# first let's load the value iteration module
using DiscreteValueIteration

# initialize the problem
mdp = evMDP()

# initialize the solver
# max_iterations: maximum number of iterations value iteration runs for (default is 100)
# belres: the value of Bellman residual used in the solver (defualt is 1e-3)
solver = ValueIterationSolver(max_iterations=100, belres=1e-3)

# initialize the policy by passing in your problem
policy = ValueIterationPolicy(mdp) 

# solve for an optimal policy
# if verbose=false, the text output will be supressed (false by default)
solve(solver, mdp, policy, verbose=true);

┌ Info: Recompiling stale cache file C:\Users\Saint Pablo\.julia\compiled\v1.0\DiscreteValueIteration\VYgAP.ji for DiscreteValueIteration [4b033969-44f6-5439-a48b-c11fa3648068]
└ @ Base loading.jl:1184


MethodError: MethodError: no method matching #evMDP#3(::Int64, ::Int64, ::Int64, ::Int64, ::Int64, ::Type{evMDP})
Closest candidates are:
  #evMDP#3(::Int64, ::Int64, ::Int64, ::Int64, !Matched::Float64, ::Any) at In[3]:15

### Simulations

In [15]:
mdp = evMDP()
mdp.tprob=1.0
sim(mdp, evState(4,1), max_steps=10) do s
    println("state is: $s")
    a = :right
    println("moving $a")
    return a
end;

MethodError: MethodError: no method matching #evMDP#3(::Int64, ::Int64, ::Int64, ::Int64, ::Int64, ::Type{evMDP})
Closest candidates are:
  #evMDP#3(::Int64, ::Int64, ::Int64, ::Int64, !Matched::Float64, ::Any) at In[3]:15